In [1]:
import sys
from pathlib import Path
sys.path.extend(['/'.join(str(Path.cwd()).split('/')[:4])])
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from classifiers.datasets import IrisDataset
from classifiers.quantum.qasvm import PseudoNormQSVM, NormQSVM
from qiskit.circuit.library import ZZFeatureMap
from qiskit.utils.quantum_instance import QuantumInstance
from qiskit.providers.aer import AerSimulator
from qiskit import BasicAer
from classifiers.optimizer import tSPSA
from classifiers.callback import CostParamStorage
from classifiers.quantum.ansatz import Circuit15

In [8]:
#ds = IrisDataset(feature_range=(0, np.pi))
#X, y = ds.sample(n=64, return_X_y=True)
from qiskit_machine_learning.datasets import ad_hoc_data
help(ad_hoc_data)
X, y, Xt, yt = ad_hoc_data(
    training_size=32,
    test_size=75,
    n=2,
    gap=0.3,
    one_hot=False
)

Help on function ad_hoc_data in module qiskit_machine_learning.datasets.ad_hoc:

ad_hoc_data(training_size, test_size, n, gap, plot_data=False, one_hot=True, include_sample_total=False)
    returns ad hoc dataset



In [9]:
var_form = Circuit15(6, reps=5)
feature_map = ZZFeatureMap(feature_dimension=2)
quantum_instance = QuantumInstance(backend=BasicAer.get_backend('qasm_simulator'), shots=2**15)
qasvm = NormQSVM(
    data=X, label=y,
    quantum_instance=quantum_instance,
    feature_map=feature_map,
    var_form=var_form
)

In [ ]:
from tqdm.notebook import tqdm
storage = CostParamStorage()
optimizer = tSPSA(maxiter=2**15, blocking=True, last_avg=16, callback=storage)
for epoch in tqdm(range(1, 2**10 + 1)):
    optimizer.step(qasvm.cost_fn, qasvm.parameters)
    """
    if storage.num_accepted() > 2 * 16 and storage.last_cost_avg(2 * 16, ignore_rejected=True) < \
            storage.last_cost_avg(16, ignore_rejected=True):
        break
        """

  0%|          | 0/1024 [00:00<?, ?it/s]

In [ ]:
%matplotlib inline
storage.plot()
plt.ylim([0., .2])

In [ ]:
qasvm.accuracy(X, y)

In [42]:
from qiskit_machine_learning.kernels import QuantumKernel

qk = QuantumKernel(feature_map, enforce_psd=False, quantum_instance = quantum_instance)
def my_kernel(x, y):
    return (np.abs(qk.evaluate([x], [y]))**2).item()

from classifiers.convex.svm import BinarySVM
svm = BinarySVM(kernel=my_kernel, k=1, C=None, mutation='REDUCED_QASVM')
svm.fit(X, y)

In [43]:
svm.accuracy(X, y)

1.0

In [ ]:
qasvm.accuracy(Xt, yt)

In [45]:
svm.accuracy(Xt, yt)

1.0